<a href="https://colab.research.google.com/github/Maagnitude/wikimovieplot-nlp-model/blob/main/wikimovieplot_nlp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3η Εργασία** στο μάθημα **Μηχανική Μάθηση και Εφαρμογές**

# **Τμήμα Πληροφορικής και Τηλεματικής - Χαροκόπειο Πανεπιστήμιο**

# **Καζάζης Γεώργιος - it214124**

Στην παρούσα εργασία θα χρησιμοποιήσουμε μεθόδους επεξεργασίας φυσικής γλώσσας.

In [6]:
import pandas as pd
import numpy as np
import os
import re
import shutil
import string

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import warnings
warnings.filterwarnings(action='ignore')

In [15]:
url = "https://raw.githubusercontent.com/Maagnitude/wikimovieplot-nlp-model/main/dataset.csv"
df = pd.read_csv(url)
df.head()

# url = "https://github.com/Maagnitude/wikimovieplot-nlp-model/blob/main/dataset.csv.gz"
# dataset = tf.keras.utils.get_file("dataset.csv.gz", url, untar=True, cache_dir='.', cache_subdir='')
# dataset_dir = os.path.join(os.path.dirname(dataset), 'dataset')
# os.listdir(dataset_dir)
# train_dir = os.path.join(dataset_dir, 'train')
# os.listdir(train_dir)


,Unnamed: 0,Release Year,Title,Origin/Ethnicity,Director,Cast,Wiki Page,Plot,Genre
0,6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,action
1,7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,comedy
2,13,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...,biography
3,14,1907,How Brown Saw the Baseball Game,American,Unknown,Unknown,https://en.wikipedia.org/wiki/How_Brown_Saw_th...,Before heading out to a baseball game at a nea...,comedy
4,15,1907,Laughing Gas,American,Edwin Stanton Porter,"Bertha Regustus, Edward Boulden",https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,The plot is that of a black woman going to the...,comedy


In [31]:
if 'Genre' in df.columns:
    print("Genre column exists.")
else:
    print("Genre column does not exist.")

Genre column exists.


In [29]:
def load_data_wiki(file_path, batch_size=32, p_train=0.65, p_val=0.15):

    # Load data with pandas
    df = pd.read_csv(file_path)

    # Shuffle data with seed 42
    df = df.sample(frac=1, random_state=42)

    # Split data into train, validation, and test sets
    n = df.shape[0]
    n_train = int(n * p_train)
    n_val = int(n * p_val)
    n_test = n - n_train - n_val

    df_train, df_val, df_test = df.iloc[:n_train], df.iloc[n_train:n_train+n_val],\
                                                          df.iloc[n_train+n_val:]
    # One-hot encode Origin/Ethnicity and Genre columns
    df_train = pd.get_dummies(df_train, columns=['Origin/Ethnicity', 'Genre'])
    df_val = pd.get_dummies(df_val, columns=['Origin/Ethnicity', 'Genre'])
    df_test = pd.get_dummies(df_test, columns=['Origin/Ethnicity', 'Genre'])

    # Clean the text
    df_train['Plot'] = df_train['Plot'].apply(lambda x: re.sub(r'[^\w\s]','',x).lower())
    df_val['Plot'] = df_val['Plot'].apply(lambda x: re.sub(r'[^\w\s]','',x).lower())
    df_test['Plot'] = df_test['Plot'].apply(lambda x: re.sub(r'[^\w\s]','',x).lower())

    # Create dictionary with keys "Title", "Origin/Ethnicity", "Plot" and "Genre"
    train_data = {'Title': df_train['Title'].values,
                  'Origin/Ethnicity': df_train.drop(['Title', 'Plot', 'Genre'], axis=1).values,
                  'Plot': df_train['Plot'].values,
                  'Genre': df_train['Genre'].values}
    val_data = {'Title': df_val['Title'].values,
                'Origin/Ethnicity': df_val.drop(['Title', 'Plot', 'Genre'], axis=1).values,
                'Plot': df_val['Plot'].values,
                'Genre': df_val['Genre'].values}
    test_data = {'Title': df_test['Title'].values,
                 'Origin/Ethnicity': df_test.drop(['Title', 'Plot', 'Genre'], axis=1).values,
                 'Plot': df_test['Plot'].values,
                 'Genre': df_test['Genre'].values}

    # Create tf.data.Dataset objects
    raw_train_ds = tf.data.Dataset.from_tensor_slices(train_data)
    raw_val_ds = tf.data.Dataset.from_tensor_slices(val_data)
    raw_test_ds = tf.data.Dataset.from_tensor_slices(test_data)

    # Batch the dataset
    raw_train_ds = raw_train_ds.batch(batch_size)
    raw_val_ds = raw_val_ds.batch(batch_size)
    raw_test_ds = raw_test_ds.batch(batch_size)

    n_labels = df_train['Genre'].shape[1]

    return raw_train_ds, raw_val_ds, raw_test_ds, n_labels

In [30]:
file_path = "https://raw.githubusercontent.com/Maagnitude/wikimovieplot-nlp-model/main/dataset.csv"
raw_train_ds, raw_val_ds, raw_test_ds, n_labels = load_data_wiki(file_path, batch_size=32, p_train=0.65, p_val=0.15)

ValueError: ignored